In [268]:
import tkinter as tk
from abc import ABCMeta
from abc import abstractmethod
import random
from tkinter import *

"""
Inherits from frame, contains a gameboard
"""
class ChessApplication(tk.Frame):
    def __init__(self,parent,size=60, color1="white", color2="brown"):
        self.board = Board()
        self.rows = 8
        self.columns = 8
        self.size = size
        self.color1 = color1
        self.color2 = color2
        self.photos = []
        self.actual_moves = () #Tuple: 1-> (row,column) 2-> list of possibilities

        canvas_width = self.columns * size
        canvas_height = self.rows * size

        tk.Frame.__init__(self, parent)
        self.canvas = tk.Canvas(self, borderwidth=0, highlightthickness=0,
                                width=canvas_width, height=canvas_height, background="bisque")
        self.canvas.bind("<Button-1>", self.callback)
        self.var = StringVar()
        label = Label( root, textvariable=self.var, relief=RAISED )

        self.var.set("Turn 0")
        label.pack(side="top",fill="both", expand=True, padx=2, pady=2)
        self.canvas.pack(side="bottom", fill="both", expand=True, padx=2, pady=2)

        # this binding will cause a refresh if the user interactively
        # changes the window size
        self.canvas.bind("<Configure>", self.refresh)
        
    def callback(self,event):
        print("callback")
        self.canvas.focus_set()
        print(str(event.x)+"  "+str(event.y))
        x = event.x
        col = 0
        while x >= 60:
            x -= 60
            col += 1
        y = event.y
        row = 0
        while y >= 60:
            y -= 60
            row += 1
        
        print("row: "+str(row)+" col: "+str(col))
        if len(self.actual_moves) != 0:
            if (row,col) in self.actual_moves[1]:
                if self.board.board[row][col] != None:
                    self.canvas.delete(str(self.board.board[row][col]))
                self.board.play(self.actual_moves[0][0],self.actual_moves[0][1],row,col)
                self.actual_moves = ()
                self.repaint()
                if len(self.board.board[row][col].get_available_moves(self.board)) == 0:
                    if self.board.whos_turn == "Enemy":
                        self.var.set("YOU WIN!")
                    else:
                        self.var.set("IA WIN :(")
                else:
                    self.var.set("Turn "+str(self.board.turn_number))
                return
                
        if self.board.board[row][col] != None:
            if (self.board.whos_turn == "Me" and not self.board.board[row][col].enemy) or (self.board.whos_turn == "Enemy" and self.board.board[row][col].enemy):
                self.paint_choices(row,col)
                return
            
        self.repaint()
        self.actual_moves = ()
        
    def repaint(self):
        xsize = 60
        ysize = 60
        self.canvas.delete("square")
        color = self.color2
        for row in range(self.rows):
            color = self.color1 if color == self.color2 else self.color2
            for col in range(self.columns):
                x1 = (col * self.size)
                y1 = (row * self.size)
                x2 = x1 + self.size
                y2 = y1 + self.size
                self.canvas.create_rectangle(x1, y1, x2, y2, outline="black", fill=color, tags="square")
                color = self.color1 if color == self.color2 else self.color2
        i = 0
        j = 0
        for row in self.board.board:
            j = 0
            for piece in row:
                if piece != None:
                    self.placepiece(str(piece), i, j)
                j += 1
            i += 1
                    
        self.canvas.tag_raise("piece")
        self.canvas.tag_lower("square")
    
    def paint_choices(self,row,col):
        moves = self.board.board[row][col].get_available_moves(self.board)
        self.actual_moves = ((row,col),moves)
        print("moves: ")
        print(moves)
        print(str(self.board))
        xsize = 60
        ysize = 60
        self.canvas.delete("square")
        color = self.color2
        for row in range(self.rows):
            color = self.color1 if color == self.color2 else self.color2
            for col in range(self.columns):
                x1 = (col * self.size)
                y1 = (row * self.size)
                x2 = x1 + self.size
                y2 = y1 + self.size
                if (row,col) in moves:
                    self.canvas.create_rectangle(x1, y1, x2, y2, outline="black", fill="blue", tags="square")
                else:
                    self.canvas.create_rectangle(x1, y1, x2, y2, outline="black", fill=color, tags="square")
                color = self.color1 if color == self.color2 else self.color2
        i = 0
        j = 0
        for row in self.board.board:
            j = 0
            for piece in row:
                if piece != None:
                    self.placepiece(str(piece), i, j)
                j += 1
            i += 1
                    
        self.canvas.tag_raise("piece")
        self.canvas.tag_lower("square")
            
    
    def start_game(self):
        #Add all pieces to board
        for i in range(len(self.board.board)):
            for j in range(len(self.board.board[i])):
                if self.board.board[i][j] != None:
                    self.addpiece(self.board.board[i][j],i,j)
    
    def addpiece(self, piece, row=0, column=0):
        '''Add a piece to the playing board'''
        photo = self.get_image_from_name(piece)
        self.photos.append(photo)
        self.canvas.create_image(0,0, image=photo, tags=(str(piece), "piece"), anchor="c")
        self.placepiece(str(piece), row, column)
        
    def get_image_from_name(self,piece):
        if isinstance(piece,Pawn):
            if piece.enemy:
                print("I've created a Pawn")
                return tk.PhotoImage(file="Chess_pdt60.gif")
            else:
                return tk.PhotoImage(file="Chess_plt60.gif") 
        if isinstance(piece,Tower):
            if piece.enemy:
                return tk.PhotoImage(file="Chess_rdt60.gif")
            else:
                return tk.PhotoImage(file="Chess_rlt60.gif")
        if isinstance(piece,Knight):
            if piece.enemy:
                return tk.PhotoImage(file="Chess_ndt60.gif")
            else:
                return tk.PhotoImage(file="Chess_nlt60.gif")
        if isinstance(piece,Bishop):
            if piece.enemy:
                return tk.PhotoImage(file="Chess_bdt60.gif")
            else:
                return tk.PhotoImage(file="Chess_blt60.gif")
        if isinstance(piece,King):
            if piece.enemy:
                return tk.PhotoImage(file="Chess_kdt60.gif")
            else:
                return tk.PhotoImage(file="Chess_klt60.gif")
        if isinstance(piece,Queen):
            if piece.enemy:
                return tk.PhotoImage(file="Chess_qdt60.gif")
            else:
                return tk.PhotoImage(file="Chess_qlt60.gif")
        print("something went wrong")

    def placepiece(self, name, row, column):
        '''Place a piece at the given row/column'''
        x0 = (column * self.size) + int(self.size/2)
        y0 = (row * self.size) + int(self.size/2)
        self.canvas.coords(name, x0, y0)

    def refresh(self, event):
        '''Redraw the board, possibly in response to window being resized'''
        xsize = int((event.width-1) / self.columns)
        ysize = int((event.height-1) / self.rows)
        self.size = min(xsize, ysize)
        self.canvas.delete("square")
        color = self.color2
        for row in range(self.rows):
            color = self.color1 if color == self.color2 else self.color2
            for col in range(self.columns):
                x1 = (col * self.size)
                y1 = (row * self.size)
                x2 = x1 + self.size
                y2 = y1 + self.size
                self.canvas.create_rectangle(x1, y1, x2, y2, outline="black", fill=color, tags="square")
                color = self.color1 if color == self.color2 else self.color2
        i = 0
        j = 0
        for row in self.board.board:
            j = 0
            for piece in row:
                if piece != None:
                    self.placepiece(str(piece), i, j)
                j += 1
            i += 1
                    
        self.canvas.tag_raise("piece")
        self.canvas.tag_lower("square")


In [ ]:
from PIL import Image, ImageTk
root = tk.Tk()
root.resizable(False,False)
root.title("CHESS")
board = ChessApplication(root)
board.start_game()
board.pack(side="top", fill="both", expand="true", padx=4, pady=4)
root.mainloop()

I've created a Pawn
I've created a Pawn
I've created a Pawn
I've created a Pawn
I've created a Pawn
I've created a Pawn
I've created a Pawn
I've created a Pawn
callback
226  372
row: 6 col: 3


Tower0 Knight2 Bishop4 Queen6 King7 Bishop5 Knight3 Tower1 
Pawn8 Pawn9 Pawn10 Pawn11 Pawn12 Pawn13 Pawn14 Pawn15 
None None None None None None None None 
None None None None None None None None 
None None None None None None None None 
None None None Pawn27 None None None None 
Pawn24 Pawn25 Pawn26 None Pawn28 Pawn29 Pawn30 Pawn31 
Tower16 Knight18 Bishop20 Queen22 King23 Bishop21 Knight19 Tower17 



Tower0 Knight2 Bishop4 Queen6 King7 Bishop5 Knight3 Tower1 
Pawn8 Pawn9 Pawn10 Pawn11 Pawn12 Pawn13 Pawn14 Pawn15 
None None None None None None None None 
None None None None None None None None 
None None None None None None None None 
None None None None None None None None 
Pawn24 Pawn25 Pawn26 Pawn27 Pawn28 Pawn29 Pawn30 Pawn31 
Tower16 Knight18 Bishop20 Queen22 King23 Bishop21 Knight19 Towe

In [267]:
class Board:
    def __init__(self,who_start="Me"):
        self.rows = 8
        self.column = 8
        self.turn_number = 0
        self._board_states = [] #Position 0 equals to board at the start of the game, position 1 after first move,...
        self._board = [[None for y in range(self.column)] for x in range(self.rows)]
        self.fill_board()
        self._board_states.append((self._board[:][:],[],[]))
        self.whos_turn = who_start
        
    def fill_board(self):
        self._board[0][0] = Tower(True,0,0,0)
        self._board[0][7] = Tower(True,0,7,1)
        self._board[0][1] = Knight(True,0,1,2)
        self._board[0][6] = Knight(True,0,6,3)
        self._board[0][2] = Bishop(True,0,2,4)
        self._board[0][5] = Bishop(True,0,5,5)
        self._board[0][3] = Queen(True,0,3,6)
        self._board[0][4] = King(True,0,4,7)
        for i in range(0,8):
            self._board[1][i] = Pawn(True,1,i,8+i)
        
        self._board[7][0] = Tower(False,7,0,16)
        self._board[7][7] = Tower(False,7,7,17)
        self._board[7][1] = Knight(False,7,1,18)
        self._board[7][6] = Knight(False,7,6,19)
        self._board[7][2] = Bishop(False,7,2,20)
        self._board[7][5] = Bishop(False,7,5,21)
        self._board[7][3] = Queen(False,7,3,22)
        self._board[7][4] = King(False,7,4,23)
        for i in range(0,8):
            self._board[6][i] = Pawn(False,6,i,i+24)
        
        
    def __str__(self):
        str_board = ""
        for row in range(self.rows):
            for piece in range(self.column):
                str_board += str(self._board[row][piece]) +" "
            str_board += "\n"
        return str_board
        
    @property
    def board(self):
        return self._board
    
    def is_king_checked(self):
        for row in range(self.rows):
            for piece in self._board[row]:
                if piece != None:
                    if piece.is_checking(self):
                        return True
        return False
    
    def undo(self):
        del self._board_states[-1]
        self._board = self._board_states[-1][0]
        self.turn_number -= 1
        self.whos_turn = "Me" if self.whos_turn == "Enemy" else "Enemy"
        
    def play(self,row_from,column_from,row_to,column_to):
        self.turn_number += 1
        next_board = [[None for y in range(self.column)] for x in range(self.rows)]
        for i in range(len(self._board)):
            for j in range(len(self._board[i])):
                if self._board[i][j] != None:
                    next_board[i][j] = self._board[i][j].__class__(False,0,0,0)
                    next_board[i][j].copy(self._board[i][j])
                
        piece = next_board[row_from][column_from]
        next_board[row_from][column_from] = None
        allies_dead = []
        enemies_dead = []
        if next_board[row_to][row_to] != None:
            if next_board[row_to][column_to].enemy != piece.enemy:
                if next_board[row_to][column_to].enemy:
                    enemies_dead.append(next_board[row_to][column_to])
                else:
                    allies_dead.append(next_board[row_to][column_to])
        next_board[row_to][column_to] = piece
        piece.move(row_to,column_to,self.turn_number)
        if len(self._board_states) != 0:
            self._board_states.append((next_board,self._board_states[-1][1]+allies_dead[:],self._board_states[-1][2]+enemies_dead[:]))
        else:
            self._board_states.append((next_board,allies_dead,enemies_dead))
        self._board = self._board_states[-1][0]
        self.whos_turn = "Me" if self.whos_turn == "Enemy" else "Enemy"
        
    def is_checking(self,enemy):
        for row in self._board:
            for piece in row:
                if piece != None:
                    if piece.is_checking(self) and enemy == piece.enemy:
                        return True
        return False
    
    def is_checked_mate(self,enemy):
        for row in self._board:
            for piece in row:
                if piece != None:
                    if len(piece.get_available_moves()) != 0:
                        return False
        return True
    
    def evaluate(self):
        ally_points = 0
        enemy_points = 0
        for row in self._board:
            for piece in row:
                if piece != None:
                    if piece.enemy:
                        enemy_points += piece.get_own_value()
                    else:
                        ally_points += piece.get_own_value()
        return (ally_points,enemy_points)
    
    

In [256]:
b = Board()
print(str(b))
b.play(6,0,5,0)
print("\n"+str(b))
b2 = Board()
b2._board = b._board_states[0][0]
print(b2)

Tower0 Knight2 Bishop4 Queen6 King7 Bishop5 Knight3 Tower1 
Pawn8 Pawn9 Pawn10 Pawn11 Pawn12 Pawn13 Pawn14 Pawn15 
None None None None None None None None 
None None None None None None None None 
None None None None None None None None 
None None None None None None None None 
Pawn24 Pawn25 Pawn26 Pawn27 Pawn28 Pawn29 Pawn30 Pawn31 
Tower16 Knight18 Bishop20 Queen22 King23 Bishop21 Knight19 Tower17 


Tower0 Knight2 Bishop4 Queen6 King7 Bishop5 Knight3 Tower1 
Pawn8 Pawn9 Pawn10 Pawn11 Pawn12 Pawn13 Pawn14 Pawn15 
None None None None None None None None 
None None None None None None None None 
None None None None None None None None 
Pawn24 None None None None None None None 
None Pawn25 Pawn26 Pawn27 Pawn28 Pawn29 Pawn30 Pawn31 
Tower16 Knight18 Bishop20 Queen22 King23 Bishop21 Knight19 Tower17 

Tower0 Knight2 Bishop4 Queen6 King7 Bishop5 Knight3 Tower1 
Pawn8 Pawn9 Pawn10 Pawn11 Pawn12 Pawn13 Pawn14 Pawn15 
None None None None None None None None 
None None None None None None Non

In [240]:
class Piece(object):
    def __init__(self,enemy,row,column,id_piece):
        self.enemy = enemy 
        self.row = row
        self.column = column
        self.moved = False
        self.id_piece = id_piece

    def move(self,row,column,turn):
        self.row = row
        self.column = column
        self.moved = True
    
    def get_available_moves(self,board):
        raise NotImplementedError()
        
    def is_checking(self,board):
        raise NotImplementedError()
        
    def get_own_value(self):
        raise NotImplementedError()
        
    def __str__(self):
        return "Not a piece"

In [241]:

class Tower(Piece):
    def __init__(self, enemy, row, column,id_piece):
        super().__init__(enemy,row,column,id_piece)
   
    def copy(self,piece):
        self.enemy = piece.enemy
        self.row = piece.row
        self.column = piece.column
        self.id_piece = piece.id_piece
        self.moved = piece.moved

    def get_available_moves(self,board):
        available_moves = []
        matrix_board = board.board
        for i in range(self.row-1,-1,-1):
            if matrix_board[i][self.column] == None:
                available_moves.append((i,self.column))
            elif matrix_board[i][self.column].enemy != self.enemy:
                available_moves.append((i,self.column))
                break
            elif matrix_board[i][self.column].enemy == self.enemy:
                break
        for i in range(self.row+1,8):
            if matrix_board[i][self.column] == None:
                available_moves.append((i,self.column))
            elif matrix_board[i][self.column].enemy != self.enemy:
                available_moves.append((i,self.column))
                break
            elif matrix_board[i][self.column].enemy == self.enemy:
                break
        for i in range(self.column-1,-1,-1):
            if matrix_board[self.row][i] == None:
                available_moves.append((self.row,i))
            elif matrix_board[self.row][i].enemy != self.enemy:
                available_moves.append((self.row,i))
                break
            elif matrix_board[self.row][i].enemy == self.enemy:
                break
        for i in range(self.column+1,8):
            if matrix_board[self.row][i] == None:
                available_moves.append((self.row,i))
            elif matrix_board[self.row][i].enemy != self.enemy:
                available_moves.append((self.row,i))
                break
            elif matrix_board[self.row][i].enemy == self.enemy:
                break
        
        #Test if being checked
        legal_moves = []
        for move in range(len(available_moves)):
            board.play(self.row,self.column,move[0],move[1])
            if board.is_checking(not self.enemy):
                legal_moves.append(move)
            board.undo()
        return legal_moves
    
    def is_checking(self,brd):
        board = brd.board
        for i in range(self.row-1,-1,-1):
            if board[i][self.column] != None:
                if board[i][self.column].enemy != self.enemy:
                    if isinstance(board[i][self.column],King):
                        return True
                    else:
                        break
                elif board[i][self.column].enemy == self.enemy:
                    break
        for i in range(self.row+1,8):
            if board[i][self.column] != None:
                if board[i][self.column].enemy != self.enemy:
                    if isinstance(board[i][self.column],King):
                        return True
                    else:
                        break
                elif board[i][self.column].enemy == self.enemy:
                    break
        for i in range(self.column-1,-1,-1):
            if board[self.row][i] != None:
                if board[self.row][i].enemy != self.enemy:
                    if isinstance(board[self.row][i],King):
                        return True
                    else:
                        break
                elif board[self.row][i].enemy == self.enemy:
                    break
        for i in range(self.column+1,8):
            if board[self.row][i] != None:
                if board[self.row][i].enemy != self.enemy:
                    if isinstance(board[self.row][i],King):
                        return True
                    else:
                        break
                elif board[self.row][i].enemy == self.enemy:
                    break
        return False
    
    def __str__(self):
        return "Tower"+str(self.id_piece)
    
    def get_own_value(self):
        if not enemy:
            return 50
        else:
            return -50
        

In [259]:
class Bishop(Piece):
    def __init__(self, enemy, row, column,id_piece):
        super().__init__(enemy,row,column,id_piece)
        
    def copy(self,piece):
        self.enemy = piece.enemy
        self.row = piece.row
        self.column = piece.column
        self.id_piece = piece.id_piece
        self.moved = piece.moved

    def get_available_moves(self,board):
        available_moves = []
        matrix_board = board.board
        for i in range(self.row-1,-1,-1):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] == None:
                available_moves.append((i,self.column+(self.row-i)))
            elif matrix_board[i][self.column+(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column+(self.row-i)))
                break
            elif matrix_board[i][self.column+i].enemy == self.enemy:
                break
        for i in range(self.row-1,-1,-1):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] == None:
                available_moves.append((i,self.column-(self.row-i)))
            elif matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column-(self.row-i)))
                break
            elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                break
        for i in range(self.row+1,8):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] == None:
                available_moves.append((i,self.column+(self.row-i)))
            elif matrix_boardd[i][self.column+(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column+(self.row-i)))
                break
            elif matrix_board[i][self.column+(self.row-i)].enemy == self.enemy:
                break
        for i in range(self.row+1,8):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] == None:
                available_moves.append((i,self.column-(self.row-i)))
            elif matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column-(self.row-i)))
                break
            elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                break
        
        #Test if being checked
        legal_moves = []
        for move in range(len(available_moves)):
            board.play(self.row,self.column,move[0],move[1])
            if board.is_checking(not self.enemy):
                legal_moves.append(move)
            board.undo()
        return legal_moves
    
    def __str__(self):
        return "Bishop"+str(self.id_piece)
    
    def get_own_value(self):
        if not enemy:
            return 30
        else:
            return -30
        
    def is_checking(self,brd):
        matrix_board = brd.board
        for i in range(self.row-1,-1,-1):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] != None:
                if matrix_board[i][self.column+(self.row-i)].enemy != self.enemy: 
                    if isinstance(matrix_board[i][self.column+(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column+(self.row-i)].enemy == self.enemy:
                    break
        for i in range(self.row-1,-1,-1):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] != None:
                if matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                    if isinstance(matrix_board[i][self.column-(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                    break
        for i in range(self.row+1,8):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] != None:
                if matrix_board[i][self.column+(self.row-i)].enemy != self.enemy:
                    if isinstance(matrix_board[i][self.column+(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column+(self.row-i)].enemy == self.enemy:
                    break
        for i in range(self.row+1,8):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] != None:
                if matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                    if isinstance(matrix_board[i][self.column-(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                    break

class King(Piece):
    def __init__(self, enemy, row, column,id_piece):
        Piece.__init__(self,enemy,row,column,id_piece)
        
    def copy(self,piece):
        self.enemy = piece.enemy
        self.row = piece.row
        self.column = piece.column
        self.id_piece = piece.id_piece
        self.moved = piece.moved
        
    def get_available_moves(self,board):
        available_moves = []
        matrix_board = board.board
        if self.row > board.rows:
            #Standard movement
            if matrix_board[self.row+1][self.column] == None or matrix_board[self.row+1][self.column].enemy != self.piece:
                available_moves.append((self.row+1,self.column))
            if self.column > board.column:
                if matrix_board[self.row+1][self.column+1] == None or matrix_board[self.row+1][self.column+1].enemy != self.piece:
                    available_moves.append((self.row+1,self.column+1))
            if self.column > 0:
                if matrix_board[self.row+1][self.column-1] == None or matrix_board[self.row+1][self.column-1].enemy != self.piece:
                    available_moves.append((self.row+1,self.column-1))
        if self.row > 0:
            #Standard movement
            if matrix_board[self.row-1][self.column] == None or matrix_board[self.row-1][self.column].enemy != self.piece:
                available_moves.add((self.row-1,self.column))
            if self.column > board.column:
                if matrix_board[self.row-1][self.column+1] == None or matrix_board[self.row-1][self.column+1].enemy != self.piece:
                    available_moves.append((self.row-1,self.column+1))
            if self.column > 0:
                if matrix_board[self.row-1][self.column-1] == None or matrix_board[self.row-1][self.column-1].enemy != self.piece:
                    available_moves.append((self.row-1,self.column-1))
        if self.column > 0:
            if matrix_board[self.row][self.column-1] == None or matrix_board[self.row][self.column-1].enemy != self.piece:
                available_moves.append((self.row,self.column-1))
            #Test enroque
            if self.moved == False:
                not_free = False
                for i in range(self.column-1,0,-1):
                    if matrix_board[self.row][i] != None:
                        not_free = True
                        break
                if isinstance(matrix_board[self.row][0],Tower) and not_free:
                    if matrix_board[self.row][0].moved == False:
                        available_moves.append(self.row,2)
                not_free = False
                for i in range(self.column+1,8):
                    if matrix_board[self.row][i] != None:
                        not_free = True
                        break
                if isinstance(matrix_board[self.row][7],Tower) and not_free:
                    if matrix_board[self.row][7].moved == False:
                        available_moves.append(self.row,6)
                        
        #Test if being checked
        legal_moves = []
        for move in range(len(available_moves)):
            board.play(self.row,self.column,move[0],move[1])
            if board.is_checking(not self.enemy):
                legal_moves.append(move)
            board.undo()
        return legal_moves
    
    def is_checking(self,board):
        return False
    
    def __str__(self):
        return "King"+str(self.id_piece)
    
    def get_own_value(self):
        if not enemy:
            return 900
        else:
            return -900
        
class Pawn(Piece):
    def __init__(self, enemy, row , column,id_piece):
        super().__init__(enemy,row,column,id_piece)
        self.promoted = False
        self.last_turn_moved = 0
    
    def copy(self,piece):
        self.enemy = piece.enemy
        self.row = piece.row
        self.column = piece.column
        self.id_piece = piece.id_piece
        self.moved = piece.moved
        self.promoted = piece.promoted
        self.last_turn_moved = piece.last_turn_moved
        
    def get_available_moves(self,board):
        available_moves = []
        matrix_board = board.board
        if self.enemy:
            if self.row < board.rows:
                #Standard movement
                if matrix_board[self.row+1][self.column] == None:
                    available_moves.append((self.row+1,self.column))
                    if self.moved == False:
                        if self.row+1 < board.rows:
                            if matrix_board[self.row+2][self.column] == None:
                                available_moves.append((self.row+2,self.column))
                #Kill piece
                if self.column < board.column:
                    if matrix_board[self.row+1][self.column+1] != None:
                        if matrix_board[self.row+1][self.column+1].enemy:
                            available_moves.append((self.row+1,self.column+1))
                    if isinstance(matrix_board[self.row][self.column+1],Pawn):
                        if matrix_board[self.row][self.column+1].promoted:
                            if matrix_board[self.row+1][self.column+1] == None:
                                available_moves.append((self.row+1,self.column+1))
                if self.column > 0:
                    if matrix_board[self.row+1][self.column-1] != None:
                        if matrix_board[self.row+1][self.column-1].enemy:
                            available_moves.append((self.row+1,self.column-1))
                    if isinstance(matrix_board[self.row][self.column-1],Pawn):
                        if matrix_board[self.row][self.column-1].promoted:
                            if matrix_board[self.row+1][self.column-1] == None:
                                available_moves.append((self.row+1,self.column-1))
        else:
            if self.row > 0:
                #Standard movement
                if matrix_board[self.row-1][self.column] == None:
                    available_moves.append((self.row-1,self.column))
                    if self.moved == False:
                        if self.row-1 < board.rows:
                            if matrix_board[self.row-2][self.column] == None:
                                available_moves.append((self.row-2,self.column))
                #Kill piece
                if self.column < board.column:
                    if matrix_board[self.row-1][self.column+1] != None:
                        if matrix_board[self.row-1][self.column+1].enemy:
                            available_moves.append((self.row-1,self.column+1))
                    if isinstance(matrix_board[self.row][self.column-1],Pawn):
                        if matrix_board[self.row][self.column-1].promoted:
                            if matrix_board[self.row-1][self.column-1] == None:
                                available_moves.append((self.row-1,self.column-1))
                if self.column > 0:
                    if matrix_board[self.row-1][self.column-1] != None:
                        if matrix_board[self.row-1][self.column-1].enemy:
                            available_moves.append((self.row-1,self.column-1))
                    if isinstance(matrix_board[self.row][self.column-1],Pawn):
                        if matrix_board[self.row][self.column-1].promoted:
                            if matrix_board[self.row-1][self.column-1] == None:
                                available_moves.append((self.row-1,self.column-1))
        legal_moves = []
        for move in available_moves:
            board.play(self.row,self.column,move[0],move[1])
            if board.is_checking(not self.enemy):
                legal_moves.append(move)
            board.undo()
        return legal_moves
    
    def is_checking(self,board):
        matrix_board = board.board
        if self.row > 0:
            #Kill piece
            if self.column < 7:
                if matrix_board[self.row-1][self.column+1] != None:
                    if matrix_board[self.row-1][self.column+1].enemy != self.enemy and isinstance(matrix_board[self.row-1][self.column+1],King):
                        return True
            if self.column > 0:
                if matrix_board[self.row-1][self.column-1] != None:
                    if matrix_board[self.row-1][self.column-1].enemy != self.enemy and isinstance(matrix_board[self.row-1][self.column-1],King):
                        return True
        return False
    
    def __str__(self):
        return "Pawn"+str(self.id_piece)
    
    def move(self,row,column,turn):
        if abs(self.row-row) == 2:
            promoted = True
            last_turn_moved = turn
        super().move(row,column,turn)
    
    def get_own_value(self):
        if not enemy:
            return 10
        else:
            return -10

In [250]:
p1 = Pawn(True,0,2,2)
p2 = p1.__class__(False,0,0,0)
p2.copy(p1)
p1.move(0,3,7)
print(p2.column)
print(p1.column)

2
3


In [258]:
class Queen(Piece):
    def __init__(self, enemy, row, column,id_piece):
        super().__init__(enemy,row,column,id_piece)

    def copy(self,piece):
        self.enemy = piece.enemy
        self.row = piece.row
        self.column = piece.column
        self.id_piece = piece.id_piece
        self.moved = piece.moved
        
    def get_available_moves(self,board):
        available_moves = []
        matrix_board = board.board
        #Act as a bishop
        for i in range(self.row-1,-1,-1):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] == None:
                available_moves.append((i,self.column+(self.row-i)))
            elif matrix_board[i][self.column+(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column+(self.row-i)))
                break
            elif matrix_board[i][self.column+i].enemy == self.enemy:
                break
        for i in range(self.row-1,-1,-1):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] == None:
                available_moves.append((i,self.column-(self.row-i)))
            elif matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column-(self.row-i)))
                break
            elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                break
        for i in range(self.row+1,8):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] == None:
                available_moves.append((i,self.column+(self.row-i)))
            elif matrix_boardd[i][self.column+(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column+(self.row-i)))
                break
            elif matrix_board[i][self.column+(self.row-i)].enemy == self.enemy:
                break
        for i in range(self.row+1,8):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] == None:
                available_moves.append((i,self.column-(self.row-i)))
            elif matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                available_moves.append((i,self.column-(self.row-i)))
                break
            elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                break
        
        #Act as a tower
        for i in range(self.row-1,-1,-1):
            if matrix_board[i][self.column] == None:
                available_moves.append((i,self.column))
            elif matrix_board[i][self.column].enemy != self.enemy:
                available_moves.append((i,self.column))
                break
            elif matrix_board[i][self.column].enemy == self.enemy:
                break
        for i in range(self.row+1,8):
            if matrix_board[i][self.column] == None:
                available_moves.append((i,self.column))
            elif matrix_board[i][self.column].enemy != self.enemy:
                available_moves.append((i,self.column))
                break
            elif matrix_board[i][self.column].enemy == self.enemy:
                break
        for i in range(self.column-1,-1,-1):
            if matrix_board[self.row][i] == None:
                available_moves.append((self.row,i))
            elif matrix_board[self.row][i].enemy != self.enemy:
                available_moves.append((self.row,i))
                break
            elif matrix_board[self.row][i].enemy == self.enemy:
                break
        for i in range(self.column+1,8):
            if matrix_board[self.row][i] == None:
                available_moves.append((self.row,i))
            elif matrix_board[self.row][i].enemy != self.enemy:
                available_moves.append((self.row,i))
                break
            elif matrix_board[self.row][i].enemy == self.enemy:
                break
                
        #Test if being checked
        legal_moves = []
        for move in range(len(available_moves)):
            board.play(self.row,self.column,move[0],move[1])
            if board.is_checking(not self.enemy):
                legal_moves.append(move)
            board.undo()
        return legal_moves
    
    def __str__(self):
        return "Queen"+str(self.id_piece)
    
    def get_own_value(self):
        if not enemy:
            return 90
        else:
            return -90
        
    def is_checking(self,brd):
        board = brd.board
        matrix_board = board
        for i in range(self.row-1,-1,-1):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] != None:
                if matrix_board[i][self.column+(self.row-i)].enemy != self.enemy: 
                    if isinstance(matrix_board[i][self.column+(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column+(self.row-i)].enemy == self.enemy:
                    break
        for i in range(self.row-1,-1,-1):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] != None:
                if matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                    if isinstance(matrix_board[i][self.column-(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                    break
        for i in range(self.row+1,8):
            if self.column+(self.row-i) >= 7:
                break
            if matrix_board[i][self.column+(self.row-i)] != None:
                if matrix_board[i][self.column+(self.row-i)].enemy != self.enemy:
                    if isinstance(matrix_board[i][self.column+(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column+(self.row-i)].enemy == self.enemy:
                    break
        for i in range(self.row+1,8):
            if self.column-(self.row-i) <= 0:
                break
            if matrix_board[i][self.column-(self.row-i)] != None:
                if matrix_board[i][self.column-(self.row-i)].enemy != self.enemy:
                    if isinstance(matrix_board[i][self.column-(self.row-i)],King):
                        return True
                    else:
                        break
                elif matrix_board[i][self.column-(self.row-i)].enemy == self.enemy:
                    break
        #Tower           
        for i in range(self.row-1,-1,-1):
            if board[i][self.column] != None:
                if board[i][self.column].enemy != self.enemy:
                    if isinstance(board[i][self.column],King):
                        return True
                    else:
                        break
                elif board[i][self.column].enemy == self.enemy:
                    break
        for i in range(self.row+1,8):
            if board[i][self.column] != None:
                if board[i][self.column].enemy != self.enemy:
                    if isinstance(board[i][self.column],King):
                        return True
                    else:
                        break
                elif board[i][self.column].enemy == self.enemy:
                    break
        for i in range(self.column-1,-1,-1):
            if board[self.row][i] != None:
                if board[self.row][i].enemy != self.enemy:
                    if isinstance(board[self.row][i],King):
                        return True
                    else:
                        break
                elif board[self.row][i].enemy == self.enemy:
                    break
        for i in range(self.column+1,8):
            if board[self.row][i] != None:
                if board[self.row][i].enemy != self.enemy:
                    if isinstance(board[self.row][i],King):
                        return True
                    else:
                        break
                elif board[self.row][i].enemy == self.enemy:
                    break
        return False
        
        
class Knight(Piece):
    def __init__(self, enemy, row , column,id_piece):
        super().__init__(enemy,row,column,id_piece)
        
    def copy(self,piece):
        self.enemy = piece.enemy
        self.row = piece.row
        self.column = piece.column
        self.id_piece = piece.id_piece
        self.moved = piece.moved

    def get_available_moves(self,board):
        available_moves = []
        matrix_board = board.board
        for n_row in range(-2,2):
            for n_col in range(-2,2):
                if n_row != 0 and n_col != 0 and n_row != n_col:
                    if self.row+n_row >= 0 and self.row+n_row <= 7 and self.column+n_col >= 0 and self.column+n_col <= 7:
                        if matrix_board[self.row+n_row][self.column+n_col] != None:
                            if matrix_board[self.row+n_row][self.column+n_col].enemy != self.enemy:
                                available_moves.append((self.row+n_row,self.column+n_col))
                        else:
                            available_moves.append((n_row,n_col))

        #Test if being checked
        legal_moves = []
        for move in range(len(available_moves)):
            board.play(self.row,self.column,move[0],move[1])
            if board.is_checking(not self.enemy):
                legal_moves.append(move)
            board.undo()
        return legal_moves
    
    def __str__(self):
        return "Knight"+str(self.id_piece)
    
    def get_own_value(self):
        if not enemy:
            return 30
        else:
            return -30
        
    def is_checking(self,brd):
        board = brd.board
        for n_row in range(-2,2):
            for n_col in range(-2,2):
                if n_row != 0 and n_col != 0 and n_row != n_col:
                    if self.row+n_row >= 0 and self.row+n_row <= 7 and self.column+n_col >= 0 and self.column+n_col <= 7:
                        if board[n_row][n_col] != None:
                            if board[n_row][n_col].enemy != self.enemy and isinstance(board[n_row][n_col],King):
                                return True
        return False